In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="y9sYzOJUOLDmrFKSNJGq")
project = rf.workspace("kitsana-uyaphitang").project("x-ray_dataset")
version = project.version(1)
dataset = version.download("folder")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 93.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to X-ray_dataset-1 in folder:: 100%|██████████| 56163/56163 [00:25<00:00, 2204.08it/s]


In [2]:
import os
import yaml


dataset_path = "/content/X-ray_dataset-1"


train_path = os.path.join(dataset_path, "train")
val_path = os.path.join(dataset_path, "valid")


class_names = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))]
class_names.sort()


data_dict = {
    'path': dataset_path,
    'train': 'train',
    'val': 'valid',
    'nc': len(class_names),
    'names': class_names
}


yaml_path = os.path.join(dataset_path, "data.yaml")


with open(yaml_path, 'w') as file:
    yaml.dump(data_dict, file, sort_keys=False)

print(f"data.yaml created at {yaml_path}")
print("Classes detected:", class_names)

data.yaml created at /content/X-ray_dataset-1/data.yaml
Classes detected: ['Unlabeled', 'covid19', 'normal', 'pneumonia']


In [3]:
!pip install ultralytics
from ultralytics import YOLO

model = YOLO("yolov8n-cls.pt")

model.train(
    data="/content/X-ray_dataset-1",
    epochs=5,
    batch=16,
    imgsz=224

)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/X-ray_dataset-1, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, 

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7918cb358980>
curves: []
curves_results: []
fitness: 0.9850416779518127
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9700833559036255, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9850416779518127}
save_dir: PosixPath('/content/runs/classify/train')
speed: {'preprocess': 0.0010894467908827508, 'inference': 13.222901549782366, 'loss': 5.4368810901359405e-05, 'postprocess': 0.00011003874073021093}
task: 'classify'
top1: 0.9700833559036255
top5: 1.0

In [ ]:
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import torch.nn as nn

# Paths
input_folder = "/content/X-ray_dataset-1/test"
output_folder = "/content/gradcam_results"
os.makedirs(output_folder, exist_ok=True)


model = YOLO("/content/runs/classify/train/weights/best.pt")
torch_model = model.model
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_model.to(device).eval()


for param in torch_model.parameters():
    param.requires_grad = True


target_layer = None
for name, module in reversed(list(torch_model.named_modules())):
    if isinstance(module, nn.Conv2d):
        target_layer = module
        break
print(" Selected target layer:", target_layer)

if target_layer is None:
    raise ValueError(" No Conv2d layer found in the model!")


cam = GradCAM(model=torch_model, target_layers=[target_layer])


image_paths = []
for root, dirs, files in os.walk(input_folder):
    for f in files:
        if f.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(root, f))

print(f" Found {len(image_paths)} test images")

if len(image_paths) == 0:
    print(" No images found! Check your input folder path.")
    print(f"Looking in: {input_folder}")
else:

    success_count = 0
    for idx, img_path in enumerate(image_paths):
        try:
            img = cv2.imread(img_path)
            if img is None:
                print(f" Could not read: {img_path}")
                continue

            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(rgb_img, (128, 128))
            resized_float = resized.astype(np.float32) / 255.0


            input_tensor = torch.tensor(resized_float).permute(2, 0, 1).unsqueeze(0).to(device)
            input_tensor.requires_grad = True

            # Prediction
            preds = model.predict(img_path, imgsz=128, verbose=False)
            pred_class = int(preds[0].probs.top1)
            pred_label = preds[0].names[pred_class]
            confidence = float(preds[0].probs.top1conf)

            # Grad-CAM
            grayscale_cam = cam(input_tensor=input_tensor,
                                targets=[ClassifierOutputTarget(pred_class)])
            grayscale_cam = grayscale_cam[0, :]

            # Overlay CAM on image
            visualization = show_cam_on_image(resized_float, grayscale_cam,
                                             use_rgb=True, image_weight=0.5)

            # Side-by-side comparison
            original_resized = cv2.resize(rgb_img, (256, 256))
            viz_resized = cv2.resize(visualization, (256, 256))
            comparison = np.hstack((original_resized, viz_resized))

            # Add labels
            cv2.putText(comparison, f"Pred: {pred_label} ({confidence:.2f})",
                       (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(comparison, "Original", (10, 240),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(comparison, "Grad-CAM", (270, 240),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


            base_name = os.path.basename(img_path)
            save_path = os.path.join(output_folder, f"gradcam_{idx}_{base_name}")
            cv2.imwrite(save_path, cv2.cvtColor(comparison, cv2.COLOR_RGB2BGR))

            success_count += 1

            if (idx + 1) % 10 == 0:
                print(f" Processed {idx + 1}/{len(image_paths)} images ({success_count} successful)")

        except Exception as e:
            print(f" Error processing {os.path.basename(img_path)}: {str(e)}")
            continue

    print(f"\n All Grad-CAM results saved in: {output_folder}")
    print(f"📊 Total images processed successfully: {success_count}/{len(image_paths)}")

# Clean up
del cam
torch.cuda.empty_cache()

✅ Selected target layer: Conv2d(256, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)
✅ Found 444 test images
✅ Processed 10/444 images (10 successful)
✅ Processed 20/444 images (20 successful)
✅ Processed 30/444 images (30 successful)
✅ Processed 40/444 images (40 successful)
✅ Processed 50/444 images (50 successful)
✅ Processed 60/444 images (60 successful)
✅ Processed 70/444 images (70 successful)
✅ Processed 80/444 images (80 successful)
✅ Processed 90/444 images (90 successful)
✅ Processed 100/444 images (100 successful)
✅ Processed 110/444 images (110 successful)
✅ Processed 120/444 images (120 successful)
✅ Processed 130/444 images (130 successful)
✅ Processed 140/444 images (140 successful)
✅ Processed 150/444 images (150 successful)
✅ Processed 160/444 images (160 successful)
✅ Processed 170/444 images (170 successful)
✅ Processed 180/444 images (180 successful)
✅ Processed 190/444 images (190 successful)
✅ Processed 200/444 images (200 successful)
✅ Processed 210/444 imag